In [1]:
%run ../../common_utils.py

In [2]:
from catboost import CatBoostRegressor
import optuna
from optuna.samplers import TPESampler
from IPython.utils import io
import unidecode
from catboost import Pool

In [3]:
random_state = 1
number_of_splits = 5

In [21]:
def load_data(val_data=False,path=None):
    print(path)
    train, test, metadata = load_all_data(path=path)
    nonCategorical, categorical = get_cat_and_non_cat_data(metadata)
    all_features = list(train.columns)
    all_features.remove('price')
    numerical_features = ['area_total','area_kitchen','area_living','floor','rooms','ceiling',
        'bathrooms_shared','bathrooms_private','balconies','loggias','phones','building_id','constructed','stories']
#     categorical_to_numerical(train, ['street','address'])
#     categorical_to_numerical(test, ['street','address'])
    if not val_data:
        X_train, y_train, test_labels = pre_process_numerical(features = all_features, numerical_features = numerical_features, train = train, test = test,
                            outliers_value=7, val_data=val_data, val_split=0.2, random_state=42, scaler="std",
                            add_R="True", add_rel_height="True", droptable=[],
                            one_hot_encode=False, cat_features=categorical, drop_old=True)
        y_train_log = np.log(y_train)
        return X_train, y_train, y_train_log, test_labels
    else:
        X_train, y_train, X_test, y_test, test_labels = pre_process_numerical(features = all_features, numerical_features = numerical_features, train = train, test = test,
                            outliers_value=7, val_data=val_data, val_split=0.2, random_state=42, scaler="std",
                            add_R="True", add_rel_height="True", droptable=[],
                            one_hot_encode=False, cat_features=categorical, drop_old=True)
        y_train_log = np.log(y_train)
        return X_train, y_train, y_train_log, X_test, y_test, test_labels
X_train, y_train, y_train_log, test_labels = load_data(path='../')

../
Std


In [22]:
selected_features_catboost=['building_id','area_kitchen', 'area_living',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'phones',
       'new', 'street', 'address', 'seller', 'layout', 'condition', 'district',
       'constructed', 'stories', 'elevator_without', 'elevator_passenger',
       'material', 'parking', 'heating', 'r', 'rel_height']

X_train = X_train[selected_features_catboost]

In [23]:
X_train['street']=X_train['street'].map(lambda row: unidecode.unidecode(row))
X_train['address']=X_train['address'].map(lambda row: unidecode.unidecode(row))

In [24]:
train, test, metadata = load_all_data(path='../')
nonCategorical, categorical = get_cat_and_non_cat_data(metadata)
text_features=['street','address']
categorical.append('street')
categorical.append('address')
X_train = X_train.astype({'street':'string','address':'string','seller':'int','layout':'int','condition':'int','district':'int','material':'int','parking':'int','heating':'int'})
X_train.drop(['area_kitchen', 'area_living', 'rooms', 'ceiling',
       'bathrooms_shared', 'bathrooms_private', 'windows_court', 
       'windows_street', 'balconies', 'loggias', 'phones', 'new', 'constructed',
       'stories', 'elevator_without', 'elevator_passenger',
       'r', 'rel_height'],axis=1,inplace=True)

In [32]:
def catboost_groupKFold(number_of_splits, model, X_train, y_train, categorical_features=[],text_features=[],random_state=1):
    scores = []
        
    cv = GroupKFold(n_splits=number_of_splits)
    
    groups = X_train["building_id"]
    i = 1
    
    for train_index, test_index in cv.split(X_train, y_train, groups):
        print("starting on split ",i)
        i+=1
        X_train2, X_test = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train2, y_test = y_train.iloc[train_index], y_train.iloc[test_index]
        X_train2.drop(['building_id'], axis=1,inplace=True)
        X_test.drop(['building_id'], axis=1,inplace=True)
        
        model.fit(
            X_train2,
            y_train2,
            eval_set=[(X_test, y_test)],
            text_features=text_features,
            feature_names=list(X_train.drop(['building_id'],axis=1).columns),
            verbose=False,
            early_stopping_rounds=100,
        )
        prediction = np.exp(model.predict(X_test))
        score = root_mean_squared_log_error(prediction, np.exp(y_test))
        scores.append(score)
    return np.average(scores)

In [33]:
model = CatBoostRegressor(random_state=random_state,loss_function='RMSE', cat_features=categorical)

average_score = catboost_groupKFold(number_of_splits, model, X_train, y_train, categorical_features=[],text_features=[],random_state=random_state)

starting on split  1


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


TypeError: fit() got an unexpected keyword argument 'text_features'

In [18]:
X_train

,building_id,street,address,seller,layout,condition,district,material,parking,heating
0,-0.274350,Buninskie Luga ZhK,k2.5/2,3,1,1,11,3,1,0
1,-0.274350,Buninskie Luga ZhK,k2.5/2,3,1,1,11,3,1,0
2,-0.274350,Buninskie Luga ZhK,k2.5/2,3,1,1,11,3,1,0
3,-0.274350,Buninskie Luga ZhK,k2.5/2,3,1,1,11,3,1,0
4,-0.274350,Buninskie Luga ZhK,k2.5/2,3,1,1,11,3,1,0
...,...,...,...,...,...,...,...,...,...,...
23280,1.354502,bul'var Troitskii,3,0,1,2,10,2,1,0
23281,-1.486403,prospekt Oktiabr'skii,9,2,1,1,10,4,1,0
23282,0.328451,Rublevskoe shosse,93K2,2,2,2,7,3,1,0
23283,-0.711220,Iuzhnobutovskaia ulitsa,75,1,1,1,6,2,1,0
